In [ ]:
# Importing libraries
import matplotlib.pyplot as plt
import numpy as np
from configs import Prometheus, Prometheus_launch_conditions
from rocketpy import Environment, Flight, Rocket, SolidMotor, GenericMotor

## RocketPy Simulation


### Environment


In [ ]:
# Environment conditions
env = Environment(
    latitude=32.99028,
    longitude=-106.96972,
    date=(2022, 6, 24, 13),
    elevation=1401
)
# may want to add wind
env.set_atmospheric_model(
    type="standard_atmosphere" #TBU
)

In [ ]:
env.info()

### Motor


In [ ]:
thrust_dict = Prometheus.motor.thrust_curve
def thrust_curve(t):
    time_list = list(thrust_dict.keys())
    thrust_list = list(thrust_dict.values())
    if t < time_list[0]:
        return thrust_list[0]
    elif t > time_list[-1]:
        return 0
    else:
        return np.interp(t, time_list, thrust_list)

# rocket: nose_to_tail
motor_M1520 = SolidMotor( # TODO: convert to GenericMotor
    thrust_source=thrust_curve,
    burn_time=Prometheus.motor.burn_time,
    dry_mass=Prometheus.motor.dry_mass,
    dry_inertia=(0, 0, 0),
    center_of_dry_mass_position= 0.274, # half of the motor length
    grains_center_of_mass_position= 0.274, # half of the motor length

    grain_number=1,#TBU, just picked to give correct mass
    grain_separation=0,#TBU
    grain_density=2420,#TBU, just picked to give correct mass
    grain_outer_radius=0.033,#TBU
    grain_initial_inner_radius=0.015,#TBU
    grain_initial_height=0.57,#TBU, just picked to give correct mass

    nozzle_radius=0.02475,#TBU
    interpolation_method="linear", # the default
)

In [ ]:
motor_M1520.plots.draw()
motor_M1520.info()

### Rocket


Create the `Rocket` object


In [ ]:
Prometheus_RP = Rocket(
    radius = 0.06985, # 5.5" diameter circle
    mass = Prometheus.rocket_mass,
    inertia=(#TBU
        73.316,#TBU
        73.316,#TBU
        0.15982,#TBU
    ),
    power_off_drag=Prometheus.Cd_rocket_at_Ma,
    power_on_drag=Prometheus.Cd_rocket_at_Ma,
    center_of_mass_without_motor=1.3,#TBU
    coordinate_system_orientation="tail_to_nose",
)

Prometheus_RP.set_rail_buttons(0.69, 0.21, 60)

Prometheus_RP.add_motor(motor=motor_M1520, position=0)

Add aerodynamic surfaces


In [ ]:
nose_cone = Prometheus_RP.add_nose(
    length=0.742, # measure and update
    kind="Von Karman",
    position=2.229
)

fin_set = Prometheus_RP.add_trapezoidal_fins(
    3,
    span=0.14, #TBU - measure on rocket
    root_chord=0.331, #TBU - measure on rocket
    tip_chord=0.1395, #TBU - measure on rocket
    position=0.314,#TBU - measure on rocket
    sweep_length=0.0698, #TBU - measure on rocket
)

Add Parachutes


In [ ]:
drogue = Prometheus_RP.add_parachute(
    "Drogue",
    cd_s=1.6*np.pi*0.3048**2, # Cd = 1.6, D_chute = 24 in
    trigger="apogee",
)
main = Prometheus_RP.add_parachute(
    "Main",
    cd_s=2.2*np.pi*0.9144**2, # Cd = 2.2, D_chute = 72 in
    trigger=457.2, # 1500 ft
)

In [ ]:
Prometheus_RP.draw()

In [ ]:
Prometheus_RP.info()

### Flight


In [ ]:
# Flight
flight = Flight(
    rocket=Prometheus_RP,
    environment = env,
    rail_length = Prometheus_launch_conditions.L_launch_rail,
    inclination = Prometheus_launch_conditions.launch_angle,
    terminate_on_apogee=False,
)

In [ ]:
flight.info()
flight.plots.trajectory_3d()